# Step 1: Evidence Details


|Evidence name|System|Source of data|Frameworks|Purpose|
|---|---|---|---|---|
|FilteredAndFormattedControlEvidence|Azure Policy|Azure Policy|AzurePolicy|Get azure policy run result and standardize it.|



```
Purpose:
Get azure policy run result and standardize it.

Suggested Name:
```
FilteredAndFormattedControlEvidence
```


# Step 2: System Specific Data (a.k.a Extended Data Schema)

---

In [ ]:
##@title # System Specific Schema / Extended Schema
#@markdown ## Describe the plugins that bring the data. This can be native to the system and/or the source of data

#Current EvidenceColumns in CC
[
  {
    "PolicyDefinitionReferenceId": "Policy123",
    "PolicyDefinitionName": "RequireEncryption",
    "PolicySetDefinitionName": "DataProtectionPolicy",
    "PolicyDefinitionGroupName": "Security",
    "ComplianceState": "Non-Compliant",
    "ComplianceReasonCode": "MissingEncryption",
    "PolicyAssignmentName": "EncryptionPolicyAssignment",
    "PolicyEvaluationTime": "2024-03-15T08:00:00Z",
    "SubscriptionId": "12345678-90ab-cdef-1234-567890abcdef",
    "ResourceID": "/subscriptions/12345678-90ab-cdef-1234-567890abcdef/resourceGroups/MyResourceGroup/providers/Microsoft.Storage/storageAccounts/MyStorageAccount",
    "ResourceType": "Microsoft.Storage/storageAccounts",
    "ResourceLocation": "eastus",
    "ResourceGroup": "MyResourceGroup",
    "ResourceTags": {
      "Environment": "Production",
      "Department": "IT"
    },
    "PolicySetDefinitionOwner": "CorporateSecurityTeam",
    "PolicySetDefinitionCategory": "DataProtection",
    "PolicyDefinitionVersion": "1.0",
    "PolicySetDefinitionVersion": "1.2",
    "Action": "EnableEncryption"
  }
]





#From ListRegisters API in Azure
GET https://management.azure.com/subscriptions/12345678-90ab-cdef-1234-567890abcdef/providers/Microsoft.ContainerRegistry/registries?api-version=2023-01-01-preview



In [ ]:

#Sample Evidence From List Registries API in AZURE

{
  "value": [
    {
      "id": "/subscriptions/00000000-0000-0000-0000-000000000000/resourceGroups/myResourceGroup/providers/Microsoft.ContainerRegistry/registries/MyRegistry",
      "name": "MyRegistry",
      "type": "Microsoft.ContainerRegistry/registries",
      "location": "westus",
      "tags": {
        "key": "value"
      },
      "sku": {
        "name": "Standard",
        "tier": "Standard"
      },
      "properties": {
        "loginServer": "myRegistry.azurecr-test.io",
        "creationDate": "2021-06-15T21:38:26.1537861Z",
        "provisioningState": "Succeeded",
        "adminUserEnabled": false,
        "networkRuleSet": { #Compliance Condtional Field For This Specific Control
          "defaultAction": "Allow", # Non compliant because no network filtering is enabled
          "ipRules": []
        },
        "policies": {
          "quarantinePolicy": {
            "status": "disabled"
          },
          "trustPolicy": {
            "type": "Notary",
            "status": "disabled"
          },
          "retentionPolicy": {
            "days": 7,
            "lastUpdatedTime": "2021-06-15T21:40:12.8506835+00:00",
            "status": "disabled"
          },
          "exportPolicy": {
            "status": "enabled"
          }
        },
        "encryption": {
          "status": "disabled"
        },
        "status": null,
        "dataEndpointEnabled": false,
        "dataEndpointHostNames": [],
        "privateEndpointConnections": [],
        "publicNetworkAccess": "Enabled",
        "networkRuleBypassOptions": "AzureServices",
        "zoneRedundancy": "Disabled",
        "anonymousPullEnabled": false
      }
    }
  ]
}

# Step 3: Define the Standard Schema

In [ ]:
#Standard Schema
[
	{
  #Meta
  "System":"azure", #[azure,]
	"Source":"compliancecow",#["compliancecow","aws_config", "aws_security_hub","azure_policy", "azure_sentinel", "azure_defender"] : Please note that the source name is all in LOWER CASE

	#ResourceInformation
  "ResourceId":"/subscriptions/00/resourceGroups/MyRegistry/providers/Microsoft.ContainerRegistry/registries/MyRegistry",
	"ResourceType":"Microsoft.ContainerRegistry/registries",
	"ResourceName":"MyRegistry",
  "ResourceUrl":"",

  #Infered Compliance Conditional fields
  "PublicNetworkAccess": "Enabled", # Field won't be available in standard schema as the source of truth is not the container registry data rather the azure policy run
                                    # result. Which doesnt contain the specific field for the control.


  #Compliance Details
    "ComplianceStatus":"",# COMPLIANT/NON_COMPLIANT/NOT_DETERMINED
		"ComplianceStatusReason":"",
    "ValidationStatusCode":"" ,
    "ValidationStatusNotes":"",
		"EvaluatedTime":"",
    # "RelatedEvidenceLink":"" ,# Extended Schema record Link - Will Have the entire api response of Container Registry List
		#Editable Data
    "Action":"",
    #Tags
    "Tags": [],

		}
]

# Step 3.a: Sample Data

| System | Source        | ResourceId                                                                                                          | ResourceType                    | ResourceName | ResourceUrl | PublicNetworkAccess | ComplianceStatus | ComplianceStatusReason | ValidationStatusCode | ValidationStatusNotes | EvaluatedTime | Action | Tags |
|--------|---------------|---------------------------------------------------------------------------------------------------------------------|---------------------------------|--------------|-------------|---------------------|-------------------|------------------------|----------------------|----------------------|---------------|--------|------|
| azure  | compliancecow | /subscriptions/00/resourceGroups/MyRegistry/providers/Microsoft.ContainerRegistry/registries/MyRegistry | Microsoft.ContainerRegistry/registries | MyRegistry   |             | Enabled             |                   |                        |                      |                      |               |        |      |


# Step 4: Compliance Taxonomy




The compliance taxonomy changes based on the ControlName field, from the ControlConfig.json, which can be found inside catalog/globalcatalog/rules/AzurePolicyFilteredAndFormattedControlEvidence/ControlConfig.json.

The below table contains the taxonomy for the ControlName: containerregistriesshouldnotallowunrestrictednetworkaccessmonitoringeffect

|CompliantStatus|ValidationStatusCode|ComplianceStatusReason|ValidationStatusNotes|
|---|---|---|---|
|COMPLIANT|PUBLIC_NETWORK_ACCESS_DISABLED|Only allowed networks can access the container registry|Only Allowed Networks should be able to access Container Registry|
|NON_COMPLIANT|PUBLIC_NETWORK_ACCESS_ENABLED|All networks can access the container registry|Disable public network access to the container registry|
|NOT_DETERMINED|NOT_DETERMINED|NOT_DETERMINED|Could not Evaluate the Control Compliance Status|

# Step 5: Calculation for Compliance Percentage and Status




In [ ]:
# Calculation of Compliance Percentage
Compliance Percentage = (Count of 'COMPLIANT' records/Total records)*100

# Compliance Status 
COMPLIANT - 100%
NON_COMPLIANT - 0%<=status<100%
NOT_DETERMINED - If no resources are found in the account


# Step 6: Remediation Steps for Non-Compliance
<br>Recomendation:Change the settings in Container Registry resource Netwok settings to Selected Networks and Provide list of allowed Networks. See the documentation for a step by step guide:
https://learn.microsoft.com/en-us/azure/container-registry/container-registry-access-selected-networks
<br>Jira or some other Ticket Creation



## Step 7. Control Setup Details


| Control Details            |                                               |
|----------------------------|-----------------------------------------------|
| **RuleName**               | AzurePolicyFilteredAndFormattedControlEvidence|
| **PreRequisiteRuleNames**  | AzurePolicyRunResult                          |
| **ExtendedSchemaRuleNames**| N/A                                           |
| **ApplicationClassName**   | azureappconnector                             |